In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_rows = []

for season in range(2025, 2026):
    url = f"https://www.espn.com/nba/standings/_/season/{season}"
    driver.get(url)
    time.sleep(5)

    

    # After driver.get(url) and time.sleep(4)
    rows = driver.find_elements(By.XPATH, "//section[contains(@class, 'Standings')]//tbody/tr")

    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) < 4:
            continue

        rank = rows.index(row) + 1  # or grab it explicitly
        team_name = cols[0].text.strip().split('\n')[0]  # Get first visible line, avoids logo clutter
        win_pct = cols[3].text.strip()  # ESPN's "PCT" column is usually at index 3

        data.append({"Rank": rank, "Team": team_name, "WinPct": win_pct, "Season": season})


    soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = soup.find_all("table")

    if len(tables) < 2:
        print(f"❌ Could not find tables for {season}")
        continue

    for i, conf in enumerate(["East", "West"]):
        rows = tables[i].find_all("tr")[1:]  # Skip header row
        for rank, row in enumerate(rows, start=1):
            cols = row.find_all("td")
            if len(cols) < 4:
                continue
            # team_name = cols[0].text.strip()
            team_name_span = cols[0].find("span", class_="hide-mobile")
            team_name = team_name_span.text.strip() if team_name_span else cols[0].text.strip()
            win_pct_raw = cols[3].text.strip()
            try:
                win_pct = float(win_pct_raw)
            except:
                win_pct = None

            all_rows.append({
                "season": season,
                "conference": conf,
                "rank": rank,
                "team": team_name,
                "win_pct": win_pct
            })
    print(f"✅ Scraped season {season}")

driver.quit()

# Save to DataFrame
df_standings = pd.DataFrame(all_rows)

# Save to SQLite
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
df_standings.to_sql("team_standings", conn, if_exists="replace", index=False)
conn.close()

print("🏆 Standings saved to database!")


✅ Scraped season 2002
✅ Scraped season 2003
✅ Scraped season 2004
✅ Scraped season 2005
✅ Scraped season 2006
✅ Scraped season 2007
✅ Scraped season 2008
✅ Scraped season 2009
✅ Scraped season 2010
✅ Scraped season 2011
✅ Scraped season 2012
✅ Scraped season 2013
✅ Scraped season 2014
✅ Scraped season 2015
✅ Scraped season 2016
✅ Scraped season 2017
✅ Scraped season 2018
✅ Scraped season 2019
✅ Scraped season 2020
✅ Scraped season 2021
✅ Scraped season 2022
✅ Scraped season 2023
✅ Scraped season 2024
✅ Scraped season 2025
🏆 Standings saved to database!


In [35]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up headless Chrome
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_rows = []

# Choose your season range
for season in range(2025, 2026):
    url = f"https://www.espn.com/nba/standings/_/season/{season}"
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = soup.find_all("table")

    if len(tables) < 2:
        print(f"❌ Could not find tables for {season}")
        continue

    for i, conf in enumerate(["East", "West"]):
        rows = tables[i].find_all("tr")[1:]  # Skip header row
        for rank, row in enumerate(rows, start=1):
            cols = row.find_all("td")
            if len(cols) < 4:
                continue

            # Try to get the cleaned team name
            team_cell = cols[0]
            print(cols[0].prettify())
            team_name_tag = team_cell.select_one("span.hide-mobile a")
            team_name = team_name_tag.text.strip() if team_name_tag else team_cell.text.strip()


            # PCT (win percentage) is in column index 3
            win_pct_raw = cols[3].text.strip()
            try:
                win_pct = float(win_pct_raw)
            except:
                win_pct = None

            all_rows.append({
                "season": season,
                "conference": conf,
                "rank": rank,
                "team": team_name,
                "win_pct": win_pct
            })

    print(f"✅ Scraped season {season}")

driver.quit()

# Save to DataFrame
df_standings = pd.DataFrame(all_rows)

# Save to SQLite (optional — uncomment if needed)
# conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
# df_standings.to_sql("team_standings", conn, if_exists="replace", index=False)
# conn.close()

print("🏆 Standings saved to DataFrame successfully!")


<td class="Table__TD">
 <span class="stat-cell">
  64
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  61
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  51
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  50
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  48
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  44
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  41
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  37
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  40
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  39
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  30
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  26
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  24
 </span>
</td>

<td class="Table__TD">
 <span class="stat-cell">
  19
 </span>
</td>

<td class="Table__TD

In [29]:
df_standings['team'].unique()  # Display unique team names to verify data

array(['64', '61', '51', '50', '48', '44', '41', '37', '40', '39', '30',
       '26', '24', '19', '18'], dtype=object)

In [27]:
# Filter for 2025 season
print(df_standings[df_standings["season"] == 2025])

# Filter for 2024 season
print(df_standings[df_standings["season"] == 2024])


     season conference  rank team  win_pct
345    2025       West     1   64      NaN
346    2025       West     2   61      3.0
347    2025       West     3   51     13.0
348    2025       West     4   50     14.0
349    2025       West     5   48     16.0
350    2025       West     6   44     20.0
351    2025       West     7   41     23.0
352    2025       West     8   37     27.0
353    2025       West     9   40     24.0
354    2025       West    10   39     25.0
355    2025       West    11   30     34.0
356    2025       West    12   26     38.0
357    2025       West    13   24     40.0
358    2025       West    14   19     45.0
359    2025       West    15   18     46.0
     season conference  rank team  win_pct
330    2024       West     1   64      NaN
331    2024       West     2   50     14.0
332    2024       West     3   49     15.0
333    2024       West     4   48     16.0
334    2024       West     5   47     17.0
335    2024       West     6   47     17.0
336    2024

In [25]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")

# View table names
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql(query_tables, conn)
print("Tables in database:")
# print(tables_df)

# Load the standings table into a DataFrame
df_work = pd.read_sql("SELECT * FROM final_player_data", conn)
df_work['team_name'].unique()  # Display unique team names to verify data

# View the first few rows
# print(df_standings.head())


Tables in database:


array(['PHI', 'LAL', 'BOS', 'ORL', 'SA', 'TOR', 'SAC', 'DAL', 'WSH',
       'TAH', 'SEA', 'MIL', 'HOU', 'CHA', 'DET', 'ATL', 'MIN', 'CHI',
       'PHX', 'NY', 'GS', 'POR', 'IND', 'DEN', 'MIA', 'LAC', 'MEM', 'CLE',
       'NO', 'NJ', 'OKC', 'BKN'], dtype=object)

In [ ]:
import pandas as pd

# Example mapping — replace the numbers with your actual observed values
team_abbr = ['PHI', 'LAL', 'BOS', 'ORL', 'SA', 'TOR', 'SAC', 'DAL', 'WSH',
             'TAH', 'SEA', 'MIL', 'HOU', 'CHA', 'DET', 'ATL', 'MIN', 'CHI',
             'PHX', 'NY', 'GS', 'POR', 'IND', 'DEN', 'MIA', 'LAC', 'MEM', 'CLE',
             'NO', 'NJ', 'OKC', 'BKN']

# Placeholder values — replace with actual numbers as per your corrupted output
team_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9,
                10, 11, 12, 13, 14, 15, 16, 17, 18,
                19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
                29, 30, 31, 32]  # <-- Update these based on your scraped data

# Create the DataFrame
team_mapping_df = pd.DataFrame({
    "team_code": team_abbr,
    "team_scraped": team_numbers
})

print(team_mapping_df)


In [38]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Connect and load data
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
df = pd.read_sql("SELECT * FROM final_player_data", conn)
conn.close()

print("Shape:", df.shape)
# print("Columns:", df.columns.tolist())

Shape: (1150, 29)


In [ ]:
features = [
    'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
    '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'REB', 'BLK', 
    'DD2', 'TD3', 'PCT', 't_rank'
]

target = "is_mvp"

In [41]:
df_model = df[features + [target] + ["season", "player_id", "Name"]].dropna()

# Split full DataFrame so we still have season info
train_df, test_df = train_test_split(df_model, test_size=0.2, random_state=42, stratify=df_model[target])

# Extract X and y
X_train = train_df[features]
y_train = train_df[target]

X_test = test_df[features]
y_test = test_df[target]


In [42]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [43]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Count class occurrences
neg_count = sum(y_train == 0)
pos_count = sum(y_train == 1)

# Calculate imbalance ratio
imbalance_ratio = neg_count / pos_count




In [44]:
# print(X_train.dtypes[X_train.dtypes == 'object'])

# Convert all columns in X_train and X_test to numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Drop any rows with NaNs (caused by conversion issues)
X_train = X_train.dropna()
y_train = y_train[X_train.index]  # Align target with filtered rows

print(X_train.dtypes[X_train.dtypes == 'object'])


Series([], dtype: object)


In [45]:
from xgboost import XGBClassifier

# Train the XGBoost model
xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=imbalance_ratio  # Only if you calculated this
)
xgb.fit(X_train, y_train)


c:\Users\priya\Documents\GitHub\NBA_MVP_Predictor\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:51:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [46]:
# Predict probabilities for class 1 (MVP)
y_pred_proba = xgb.predict_proba(X_test)[:, 1]

# Add predictions to a copy of X_test
season_rankings = X_test.copy()
season_rankings['season'] = df.loc[X_test.index, 'season']
season_rankings['player_id'] = df.loc[X_test.index, 'player_id']
season_rankings['is_mvp'] = y_test
season_rankings['mvp_prob'] = y_pred_proba


In [47]:
# Pick player with highest predicted MVP probability per season
predicted_mvp = season_rankings.sort_values('mvp_prob', ascending=False).groupby('season').head(1)


In [48]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# True MVPs per season
true_mvp = season_rankings[season_rankings['is_mvp'] == 1].groupby('season').head(1)

# Match predicted MVPs to actual MVPs
merged = pd.merge(predicted_mvp, true_mvp, on='season', suffixes=('_pred', '_true'))

# Correct prediction = same player_id
merged['correct'] = merged['player_id_pred'] == merged['player_id_true']

# Metrics
accuracy = merged['correct'].mean()
precision = precision_score(merged['correct'], [1]*len(merged))  # All predicted = 1
recall = recall_score(merged['correct'], [1]*len(merged))        # True MVPs = 1
f1 = f1_score(merged['correct'], [1]*len(merged))

print(f"🎯 Accuracy: {accuracy:.2f}")
print(f"🔍 Precision: {precision:.2f}")
print(f"📈 Recall: {recall:.2f}")
print(f"🏅 F1 Score: {f1:.2f}")


🎯 Accuracy: 0.75
🔍 Precision: 0.75
📈 Recall: 1.00
🏅 F1 Score: 0.86
